In [1]:
import sys
import os
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_path)

In [2]:
from src.bpmn_generation import BPMNGenerationGPT
import os
ROOT_DIR = r"C:\Projects\Research\SWEEP\SWEEP"

sector = "banking"
activity = "loan_risk_assessment"

bpmn_generator = BPMNGenerationGPT()
folder_path = os.path.join(ROOT_DIR, f"data\\{sector}\\{activity}")
image_name = f"{activity}_diagram.png"
original_image_path = os.path.join(folder_path, image_name)
mermaid_code = bpmn_generator.generate_bpmn(original_image_path)    
if mermaid_code:
    print(mermaid_code)
else:
    print("Failed to generate BPMN notation.")

graph TD
    A([Receive loan application]) --> B[Initial eligibility check]
    B -->|eligible| C{XORGateway}
    B -->|not eligible| Z([Application ineligible])
    C -->|eligible| D{ParallelGateway}
    D --> E[Credit history check]
    D --> F[Check applicant's income sources]
    D --> G[Appraisal of property to be mortgaged]
    E --> H{XORGateway}
    F --> H
    G --> I{XORGateway}
    H --> J[Assess applicant's credit risk]
    I -->|mortgageable| K[Appraise other collateral]
    I -->|not mortgageable| L{XORGateway}
    L --> M[Customer informed]
    M --> Z
    K --> N{ParallelGateway}
    J --> N
    N --> O[Loan risk assessment]
    O --> P([Risk assessed])


In [3]:
from src.bpmn_generation import MermaidImageGenerator

flow_image_generator = MermaidImageGenerator(mmdc_path=r"C:\Users\wmichele\AppData\Roaming\npm\mmdc.cmd")

# get the diagram name from the image path
diagram_name = image_name.split(".")[0] + "_generated.png"
result_path = os.path.join(ROOT_DIR, f"test\\results\\bpmn_generation\\{sector}\\{activity}")

generated_image_path = os.path.join(result_path, diagram_name)
success = flow_image_generator.generate_image(mermaid_code, generated_image_path)

Image generated successfully: C:\Projects\Research\SWEEP\SWEEP\test\results\value_classification\banking\loan_risk_assessment\loan_risk_assessment_diagram_generated.png


In [1]:
from src.bpmn_generation import BPMNGenerationEvaluatorGPT

evaluator = BPMNGenerationEvaluatorGPT()

equivalance, revisions = evaluator.evaluate_bpmn(original_image_path, generated_image_path)

if revisions:
    print("Evaluation Result:")
    print(f"equivalance: {equivalance}")
    
    print(revisions)
else:
    print("Failed to evaluate BPMN diagrams.")

In [5]:
def revise_and_evaluate_bpmn(original_image_path, current_mermaid_code, result_path, image_name, iteration=0, max_iterations=3):
    """
    Recursively revise and evaluate BPMN diagrams.
    
    :param original_image_path: Path to the original image
    :param current_mermaid_code: Current Mermaid code to revise
    :param result_path: Path to the folder for saving generated images
    :param image_name: Name of the image file
    :param iteration: Current iteration count
    :param max_iterations: Maximum number of iterations allowed
    :return: Tuple of (final_equivalence, final_mermaid_code, final_revisions)
    """
    if iteration >= max_iterations:
        print(f"Reached maximum iterations ({max_iterations}). Stopping revision process.")
        return False, current_mermaid_code, "Max iterations reached. Further manual review may be needed."

    # Generate image from current Mermaid code
    revised_diagram_name = f"{image_name.split('.')[0]}_generated_revised_{iteration}.png"
    generated_revised_image_path = os.path.join(result_path, revised_diagram_name)
    success = flow_image_generator.generate_image(current_mermaid_code, generated_revised_image_path)

    if not success:
        print(f"Failed to generate image for iteration {iteration}.")
        return False, current_mermaid_code, "Failed to generate image"

    # Evaluate the revised BPMN
    equivalence, revisions = evaluator.evaluate_bpmn(original_image_path, generated_revised_image_path)

    print(f"\nIteration {iteration + 1} Evaluation Result:")
    print(f"Equivalence: {equivalence}")
    print(revisions)

    if equivalence:
        return True, current_mermaid_code, revisions

    # If not equivalent, revise the BPMN
    updated_mermaid_code = bpmn_generator.revise_bpmn(revisions)

    if updated_mermaid_code:
        print(f"\nUpdated Mermaid Code (Iteration {iteration + 1}):")
        print(updated_mermaid_code)
    else:
        print(f"Failed to generate BPMN notation in iteration {iteration + 1}.")
        return False, current_mermaid_code, "Failed to generate updated BPMN notation"

    # Recursive call
    return revise_and_evaluate_bpmn(
        original_image_path, updated_mermaid_code, 
        result_path, image_name, iteration + 1, max_iterations
    )

In [6]:
if not equivalance:
    final_equivalence, final_mermaid_code, final_revisions = revise_and_evaluate_bpmn(original_image_path, mermaid_code, result_path, image_name, iteration=0, max_iterations=3)